# Module 2 - Working with Offline Feature Store 
## Features Store Dataset Extraction
**How to use Amazon SageMaker Feature Store to retrieve and share machine learning (ML) features in order to build feature sets that can be used for training a ML model.**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.m5.4xlarge`

---

## Contents

1. [Overview](#Overview)
1. [Data Query](#Data-Query)
    1. [Look in S3 console](#Look-in-S3-console)
    1. [Query in Athena console](#Query-in-Athena-console)
    1. [Query using SageMaker SDK](#Query-using-SageMaker-SDK)
1. [Extract a feature set](#Extract-a-feature-set)    
1. [Row level time travel](#Row-Level-Time-Travel)
1. [Optional queries and validations](#Optional-queries-and-validations)
    1. [Browse the set of offline store files in the S3 console](#Browse-the-set-of-offline-store-files-in-the-S3-console.)
    1. [See the Glue tables that are used for Athena queries](#See-the-Glue-tables-that-are-used-for-Athena-queries.)
    1. [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)
    1. [Count the rows in an offline store](#Count-the-rows-in-an-offline-store.)
    1. [Get a random sample of offline store rows](#Get-a-random-sample-of-offline-store-rows.)

---

## Overview

In previous module (`Module-1 Introduction to SageMaker Feature Store`), we demonstrated how to create multiple features groups inside a Amazon SageMaker Feature Store and ingest data into it.

In this notebook, we will illustrate how to retrieve the ingested features from the multiple feature groups and combine them to build feature sets that can be used to train a ML model. We will cover the following aspects:

* Look at data via S3 console (Offline feature store)
* Athena query for dataset extraction - shown via Athena console
* Athena query for dataset extraction (programmatically using SageMaker SDK)
* Extract training dataset and persist to S3
* Some additional queries to visualize a Parquet file, count the rows and get a random sample of offline store

---


## Data Query

Before starting, in order to get into Feature Store in SageMaker Studio in the left side menu you need to select the SageMaker Components and registries, then select Feature Store from the list of components and click on the Feature store button.

![Feature Store](../images/FS_0.png "Feature Store")

First, let's start by looking at the feature store data. 

Once the features groups are created (as demonstrated in `Module-1`), we should be able to see the three features groups - `orders`, `products` and `customers` from the SageMaker Studio UI.

![Features Groups](../images/FS1.png "Features Groups")

Double click on one feature group to open a new correspondent tab in Studio. Inside one particular feature group, we can check the location of the offline data by looking for s3 on the bottom right like in the following image. 

![Orders FG](../images/FS2.png "Orders FG")

### Look in S3 console

In a different tab please open S3 console in your AWS account. The location of the offline feature store data is in S3 inside the default S3 bucket and prefix will respect the following format:

s3://DEFAULT_BUCKET/sagemaker-feature-store/ACCOUNT_ID/sagemaker/REGION/offline-store/

// Ours is `sagemaker-us-west-2-119387606724`

![S3 Location](../images/S3_1.png "S3 Location")

We notice that in S3 we have a timestamp suffix added for each feature group name and this corresponds to what is displayed in Athena console as tables.

You can get the S3 URL from your environment after launching the cell referenced by: [Browse the set of offline store files in the S3 console.](#Browse-the-set-of-offline-store-files-in-the-S3-console.)

Inside each feature group, we have a `data` subdirectory followed by directories partitioned by year/month/day/hour of Parquet files.

![S3 Files](../images/S3_2.png "S3 Files")

You will can get a look into a sample Parquet file after launching the cell referenced by: [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)

---
### Query in Athena console

If it is for the first time we are launching Athena in AWS console we need to click on `Get Started` button and then before we run the first query we need to set up a query results location in Amazon S3. For simplicity, we can choose the same default SageMaker bucket that is used by Feature Store.

![Athena results location](../images/AthenaSetupMessage.png "Athena results location")

After setting the query results location, on the left panel we need to select the `AwsDataCatalog` as Data source and the `sagemaker_featurestore` as Database.

We can run now run a query for the offline feature store data in Athena. To select the entries from the orders feature group we use the following SQL query. You will need to replace the orders table name with the corresponded value from your environment.

```sql
select * from "<orders-feature-group-table-name>"
limit 20
```

![Athena Orders](../images/AthenaOrders.png "Athena Orders")

You can notice that Feature Store adds the `write_time`, `api_invocation_time` and `is_deleted` fields to the offline store.

Similarly, we can query the products and the customers feature group data by replacing the table name with the correspondent value. 

```sql
select * from "<products-feature-group-table-name>"
limit 20
select * from "<customers-feature-group-table-name>"
limit 20
```

Since our three features groups are related, we can do a join query by grouping all the information by product ID and by customer ID like here below. Please make sure you are replacing the table names with the corresponding values from your environment.

```sql
select *
FROM
  "<orders-feature-group-table-name>"
, "<products-feature-group-table-name>"  
, "<customers-feature-group-table-name>"
WHERE ("<orders-feature-group-table-name>"."customer_id" = "<customers-feature-group-table-name>"."customer_id")
AND ("<orders-feature-group-table-name>"."product_id" = "<products-feature-group-table-name>"."product_id")
limit 20
```

![Athena Join](../images/AthenaJoin.png "Athena Join")

---

### Query using SageMaker SDK

Extract the data from Feature Store using Athena query using SageMaker SDK

In [3]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
sys.path.append('..')
from utilities import Utils
from utilities import feature_store_helper

In [4]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [5]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [6]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retreive the orders, products and customers feature group names

In [7]:
# Retreive FG names
%store -r customers_feature_group_name
%store -r products_feature_group_name
%store -r orders_feature_group_name
# Retreive FG row count
%store -r customers_count
%store -r products_count
%store -r orders_count

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)

#### Build Athena join query to combine the 3 features groups - `customers`, `products` & `orders`

In [8]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

In [9]:
query_string = f'SELECT * FROM "{customers_table}", "{products_table}", "{orders_table}" ' \
               f'WHERE ("{orders_table}"."customer_id" = "{customers_table}"."customer_id") ' \
               f'AND ("{orders_table}"."product_id" = "{products_table}"."product_id")'
%store query_string
query_string

Stored 'query_string' (str)


'SELECT * FROM "fscw-customers-02-15-23-31-1644968214", "fscw-products-02-15-23-31-1644968233", "fscw-orders-02-15-23-31-1644968254" WHERE ("fscw-orders-02-15-23-31-1644968254"."customer_id" = "fscw-customers-02-15-23-31-1644968214"."customer_id") AND ("fscw-orders-02-15-23-31-1644968254"."product_id" = "fscw-products-02-15-23-31-1644968233"."product_id")'

In [10]:
output_location = f's3://{default_bucket}/{query_results}/query_results/'
print(f'Athena query output location: \n{output_location}')

Athena query output location: 
s3://sagemaker-us-west-2-119387606724/sagemaker-featurestore-workshop/query_results/


#### Check to see if data is available in offline store.

In [11]:
# Before extracting the data we need to check if the feature store was populated
offline_store_contents = None
while offline_store_contents is None:    
    customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
    products_total_record_count = Utils.get_historical_record_count(products_feature_group_name)
    orders_total_record_count = Utils.get_historical_record_count(orders_feature_group_name)
    if customers_total_record_count >= customers_count and \
        products_total_record_count >= products_count and \
        orders_total_record_count >= orders_count:
        logger.info('[Features are available in Offline Store!]')
        offline_store_contents = orders_total_record_count
    else:
        logger.info('[Waiting for data in Offline Store ...]')
        time.sleep(60)

[Features are available in Offline Store!]


#### Run Athena query and load the output as a Pandas dataframe.

In [12]:
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()
joined_df = orders_query.as_dataframe()
joined_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,...,order_id,customer_id.1,product_id.1,purchase_amount,is_reordered,event_time.2,n_days_since_last_purchase,write_time.2,api_invocation_time.2,is_deleted.2
0,C7252,0,1,2022-02-15T23:29:48.477Z,1,0,0,0,0,0,...,O48491,C7252,P12484,0.530990,1,2022-02-15T23:29:56.191Z,0.377907,2022-02-15 23:47:16.815,2022-02-15 23:43:54.000,False
1,C6033,1,1,2022-02-15T23:29:46.666Z,0,1,0,0,0,0,...,O36214,C6033,P1920,0.790792,0,2022-02-15T23:29:55.483Z,0.296512,2022-02-15 23:47:16.524,2022-02-15 23:43:59.000,False
2,C6033,1,1,2022-02-15T23:29:46.666Z,0,1,0,0,0,0,...,O71172,C6033,P12428,0.209208,1,2022-02-15T23:29:57.576Z,0.903101,2022-02-15 23:47:18.198,2022-02-15 23:43:08.000,False
3,C6033,1,1,2022-02-15T23:29:46.666Z,0,1,0,0,0,0,...,O94846,C6033,P16261,0.873465,0,2022-02-15T23:29:58.983Z,0.023256,2022-02-15 23:47:16.365,2022-02-15 23:42:42.000,False
4,C6033,1,1,2022-02-15T23:29:46.666Z,0,1,0,0,0,0,...,O4406,C6033,P9928,0.558416,0,2022-02-15T23:29:53.643Z,0.312016,2022-02-15 23:47:16.638,2022-02-15 23:43:46.000,False


In [13]:
joined_df.columns

Index(['customer_id', 'sex', 'is_married', 'event_time', 'age_18-29',
       'age_30-39', 'age_40-49', 'age_50-59', 'age_60-69', 'age_70-plus',
       'n_days_active', 'write_time', 'api_invocation_time', 'is_deleted',
       'product_id', 'event_time.1', 'category_baby_food_formula',
       'category_baking_ingredients', 'category_candy_chocolate',
       'category_chips_pretzels', 'category_cleaning_products',
       'category_coffee', 'category_cookies_cakes', 'category_crackers',
       'category_energy_granola_bars', 'category_frozen_meals',
       'category_hair_care', 'category_ice_cream_ice',
       'category_juice_nectars', 'category_packaged_cheese',
       'category_refrigerated', 'category_soup_broth_bouillon',
       'category_spices_seasonings', 'category_tea',
       'category_vitamins_supplements', 'category_yogurt', 'write_time.1',
       'api_invocation_time.1', 'is_deleted.1', 'order_id', 'customer_id.1',
       'product_id.1', 'purchase_amount', 'is_reordered', 'eve

## Extract a feature set

**Note:** This extracted feature set will be used for model training in `Module-3`

##### Drop columns which are not needed for training

In [14]:
model_df = joined_df.drop(['order_id', 
                           'customer_id', 
                           'product_id', 
                           'event_time', 
                           'write_time', 
                           'api_invocation_time', 
                           'is_deleted', 
                           'product_id.1', 
                           'event_time.1', 
                           'write_time.1', 
                           'api_invocation_time.1', 
                           'is_deleted.1', 
                           'customer_id.1', 
                           'purchase_amount',
                           'event_time.2', 
                           'n_days_since_last_purchase',
                           'write_time.2', 
                           'api_invocation_time.2', 
                           'is_deleted.2'], axis=1)

In [15]:
model_df.head(5)

,sex,is_married,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,category_baby_food_formula,...,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt,is_reordered
0,0,1,1,0,0,0,0,0,0.493151,0,...,0,0,0,0,0,0,0,0,1,1
1,1,1,0,1,0,0,0,0,0.189041,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0.189041,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,1,0,0,0,0,0.189041,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,0,0,0,0,0.189041,0,...,0,0,0,0,0,0,0,0,0,0


#### Write transformed features to local `data` directory

In [16]:
model_df.to_csv('.././data/train/transformed.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '.././data/train/transformed.csv'

#### Copy file from local to S3

In [17]:
boto3.Session().resource('s3').Bucket(default_bucket).Object(os.path.join(query_results, 'transformed.csv')).upload_file('.././data/train/transformed.csv')

FileNotFoundError: [Errno 2] No such file or directory: '.././data/train/transformed.csv'

## Row Level Time Travel

Features in different feature groups can be updated at different times. In order to build a training dataset based on specific event timestamps, we need a way to extract the feature values across all the feature group which are valid at the given timestamp using point-in-time queries. Data scientists can query for the exact set of feature values that would have been available at a specific time, without the chance of including data from beyond that time.

[Refer to this blog post for details](https://aws.amazon.com/blogs/machine-learning/build-accurate-ml-training-datasets-using-point-in-time-queries-with-amazon-sagemaker-feature-store-and-apache-spark/)

![Row Level Time Travel](../images/m2_nb1_row_level_time_travel.png "Row Level Time Travel")


Here we use a helper class containing many utility functions. The `get_features` method implements row-level time travel for us, taking in a feature set across multiple feature groups, and taking in an events dataframe with timestamps and identifiers (join keys). In this simple example, our offline store only has a single version of the data, so this is a trivial example. In a real-world example, the event timestamps would be distributed across history, and the time travel function would accurately pull the features from each feature group based on those timestamps. Note that we can either specify an explicit list of individual features, or we can use a wildcard to retrieve all features from a given feature group. `get_features` is a convenient way to gather a training dataset without having to write complicated SQL queries.

Note - This is just demonstrating the use of row-level time travel, but the rest of the workshop does not depend upon the results.

In [ ]:
#Building a dataframe of transformed order data
orders_df = pd.read_csv('.././data/transformed/orders.csv')

multi_id_events = []

#Iterating over the first 10 rows from the data frame
for ind in orders_df.head(10).index:
    event = [orders_df['event_time'][ind], orders_df['order_id'][ind],
             orders_df['product_id'][ind], orders_df['customer_id'][ind]]
    multi_id_events.append(event)
    
multi_id_df = pd.DataFrame(multi_id_events, columns=['my_event_time', 'order_id', 'product_id', 'customer_id' ])
multi_id_df.head(10)





In [ ]:
from utilities.feature_store_helper import FeatureStore
fs = FeatureStore()

# Calling the helper function to return the feature values across multiple feature groups valid for the given 
# event timestamps. Note the usage of wildcard for the product feature group.


fs.get_features(multi_id_df, 'my_event_time', 
                   features=[f'{customers_feature_group_name}:is_married',
                             f'{products_feature_group_name}:*',
                             f'{orders_feature_group_name}:purchase_amount'],
               parallel=False)


## Optional queries and validations

### Browse the set of offline store files in the S3 console.

In [18]:
customers_s3_console_url = Utils.get_offline_store_url(customers_feature_group_name)
products_s3_console_url = Utils.get_offline_store_url(products_feature_group_name)
orders_s3_console_url = Utils.get_offline_store_url(orders_feature_group_name)

logger.info('Review customers offline store partitioned data files here: '+customers_s3_console_url)
logger.info('\nReview products offline store partitioned data files here: '+products_s3_console_url)
logger.info('\nReview orders offline store partitioned data files here: '+orders_s3_console_url)

Review customers offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-west-2-119387606724?region=us-west-2&prefix=sagemaker-feature-store/119387606724/sagemaker/us-west-2/offline-store/fscw-customers-02-15-23-31-1644968214/data/

Review products offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-west-2-119387606724?region=us-west-2&prefix=sagemaker-feature-store/119387606724/sagemaker/us-west-2/offline-store/fscw-products-02-15-23-31-1644968233/data/

Review orders offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-west-2-119387606724?region=us-west-2&prefix=sagemaker-feature-store/119387606724/sagemaker/us-west-2/offline-store/fscw-orders-02-15-23-31-1644968254/data/


### See the Glue tables that are used for Athena queries.

In [19]:
customers_glue_console_url = Utils.get_glue_table_url(customers_feature_group_name)
products_glue_console_url = Utils.get_glue_table_url(products_feature_group_name)
orders_glue_console_url = Utils.get_glue_table_url(orders_feature_group_name)

logger.info('To see the customers Glue table that was created for you, go here: '+customers_glue_console_url)
logger.info('\nTo see the products Glue table that was created for you, go here: '+products_glue_console_url)
logger.info('\nTo see the orders Glue table that was created for you, go here: '+orders_glue_console_url)

To see the customers Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-west-2#table:catalog=119387606724;name=fscw-customers-02-15-23-31-1644968214;namespace=sagemaker_featurestore

To see the products Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-west-2#table:catalog=119387606724;name=fscw-products-02-15-23-31-1644968233;namespace=sagemaker_featurestore

To see the orders Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-west-2#table:catalog=119387606724;name=fscw-orders-02-15-23-31-1644968254;namespace=sagemaker_featurestore


### Examine contents of a sample offline store Parquet file.

In [20]:
# Let's have a look to the customers FG
customers_sample_filename = Utils.download_sample_offline_file(customers_feature_group_name)
logger.info('Downloaded sample Parquet file from offline store: '+customers_sample_filename+'\n')

customers_sample_df = pd.read_parquet(customers_sample_filename)
customers_sample_df.head()

Downloaded sample Parquet file from offline store: 20220215T232952Z_05gT1OqbVV1GJoz7.parquet



,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,write_time,api_invocation_time,is_deleted
0,C1262,1,1,2022-02-15T23:29:39.490Z,0,0,0,0,1,0,0.570548,2022-02-15 23:46:22.993000+00:00,2022-02-15 23:41:22+00:00,False
1,C4380,0,1,2022-02-15T23:29:44.042Z,0,0,0,0,1,0,0.363699,2022-02-15 23:46:22.993000+00:00,2022-02-15 23:41:22+00:00,False
2,C4383,0,0,2022-02-15T23:29:44.047Z,0,1,0,0,0,0,0.969863,2022-02-15 23:46:22.993000+00:00,2022-02-15 23:41:22+00:00,False
3,C2507,1,1,2022-02-15T23:29:41.292Z,1,0,0,0,0,0,0.114384,2022-02-15 23:46:22.993000+00:00,2022-02-15 23:41:22+00:00,False
4,C43,0,0,2022-02-15T23:29:37.538Z,0,0,0,0,0,1,0.039726,2022-02-15 23:46:22.993000+00:00,2022-02-15 23:41:22+00:00,False


### Count the rows in an offline store.

In [21]:
# Get the number of rows from customers FG
customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
logger.info(f'Found {customers_total_record_count:,d} records in "{customers_feature_group_name}" feature group.')

Found 10,000 records in "fscw-customers-02-15-23-31" feature group.


### Get a random sample of offline store rows.

In [22]:
# Get random rows from customers FG
customers_sample_df = Utils.sample(customers_feature_group_name, n=5)
customers_sample_df.head()

Running query :
 SELECT * FROM "fscw-customers-02-15-23-31-1644968214" tablesample bernoulli(25) limit 5
On  database: sagemaker_featurestore


,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,write_time,api_invocation_time,is_deleted
0,C2533,1,0,2022-02-15T23:29:41.329Z,0,0,0,0,0,1,0.347260,2022-02-15 23:46:23.010,2022-02-15 23:41:22.000,False
1,C9407,0,1,2022-02-15T23:29:51.649Z,0,0,0,0,1,0,0.028082,2022-02-15 23:46:23.010,2022-02-15 23:41:23.000,False
2,C8797,1,1,2022-02-15T23:29:50.759Z,0,1,0,0,0,0,0.247945,2022-02-15 23:46:23.010,2022-02-15 23:41:23.000,False
3,C98,0,0,2022-02-15T23:29:37.646Z,0,1,0,0,0,0,0.763699,2022-02-15 23:46:23.010,2022-02-15 23:41:23.000,False
4,C733,0,0,2022-02-15T23:29:38.719Z,0,0,1,0,0,0,0.117123,2022-02-15 23:46:23.010,2022-02-15 23:41:23.000,False
